In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import cv2
import os
from datetime import timedelta
import sys
from pathlib import Path 

In [2]:
def format_timedelta(td):
    """Служебная функция для классного форматирования объектов timedelta (например, 00:00:20.05)
    исключая микросекунды и сохраняя миллисекунды"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")




In [3]:
def get_saving_frames_durations(cap, saving_fps, df, delta = 5):
    """Функция, которая возвращает список длительностей, в которые следует сохранять кадры.
    df - инфо по нарушениям 
    delta - сколько секунд отступаем до нарушения
    """

    # получаем продолжительность клипа, разделив количество кадров на количество кадров в секунду
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
        
    flags = [] # метки флага нарушений
    number_of_violent = [] # номер нарушения в кадре
    descr_list = [] # список описаний кадров
    s = []  # список секунд для стоп-кадров
    end_old = 0
    for idx, row in df.iterrows():

        m_start = row.start.minute
        s_start = row.start.second

        m_end = row.end.minute
        s_end = row.end.second

         # преобразуем время нарушений в секунды    
        start = m_start*60 + s_start
        end = m_end*60 + s_end
        
        # определяем время начала нарушения           
        if start-delta < end_old:
            start_new = start
        else: 
            start_new = start-delta
            
        descr = row.description
        
        # определяем список кадров    
        for i in np.arange(start_new, end, 1 / saving_fps):
            s.append(i)
            number_of_violent.append(idx)
            if i < start:
                #  до начала нарушения флаг 0           
                flags.append(0)
                descr_list.append(None)
            else:
                flags.append(1)
                descr_list.append(descr)
        end_old = end
    return (s, number_of_violent, flags, descr_list)

In [4]:
def main(video_file, video_name, video_num, df, delta, save_dir, make_dir=False):
    """
    Функция выделяет в видео отрезки с нарушенями и за __ секунд до нарушения,
    создает кадры и сохраняет их в заданную папку.
    
    Args:
        video_file - путь до видео
        video_name - название видео
        video_num - номер видео
        df - данные по нарушениям для конкретного видео
        delta - сколько секунд отступаем до нарушения
        make_dir - сохраняем в одну папку или в разные
        save_dir - куда сохраняем
    """
    file_names = pd.DataFrame()
    
    # создаем папку для сохранения результатов
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
        
    filename = save_dir
    
    if make_dir:
        filename, _ = os.path.splitext(video_name)
        filename = os.path.join(save_dir, filename+"_opencv")    
        # создаем индивидуальную папку для сохранения результатов
        if not os.path.isdir(filename):
            os.mkdir(filename)
            
    # читать видео файл    
    cap = cv2.VideoCapture(video_file)
    # получить FPS видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    # если SAVING_FRAMES_PER_SECOND выше видео FPS, то установите его на FPS (как максимум)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # получить список длительностей для сохранения
    saving_frames_durations, number_of_violent, flags, descr_list = get_saving_frames_durations(cap, saving_frames_per_second, df, delta)
    # запускаем цикл
    count = 0
    save_frame_count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # выйти из цикла, если нет фреймов для чтения
            break
        # получаем продолжительность, разделив количество кадров на FPS
        frame_duration = count / fps
        try:
            # получить самую раннюю продолжительность для сохранения
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # список пуст, все кадры длительности сохранены
            break
        if frame_duration >= closest_duration:

            # если ближайшая длительность меньше или равна длительности кадра,
            # затем сохраняем фрейм
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))           
            
            count_violation = number_of_violent[save_frame_count]
            flag = flags[save_frame_count]
            descr = descr_list[save_frame_count]
            frame_name = f"video_{video_num}_frame_{frame_duration_formatted}_flag_{flag}_violation_{count_violation}.jpg"
#             print(frame_name)

            cv2.imwrite(os.path.join(filename, frame_name), frame) 
            file_df = pd.DataFrame({'video_name': video_name,
                                 'video_num': video_num,
                                 'image_name': frame_name,
                                 'frame_duration': frame_duration_formatted,
                                 'target': flag,
                                 'description': descr}, index=[0])
            file_names = pd.concat([file_names, file_df], ignore_index=True)
    
#             file_names.append(frame_name)
            save_frame_count += 1
            # удалить точку продолжительности из списка, так как эта точка длительности уже сохранена
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # увеличить количество кадров
        count += 1
    return file_names

In [5]:
info_df = pd.read_excel('разметка.xlsx')

In [6]:
info_df

,videofile,start,end,description
0,0000000_00000020240221082923_0001_IMP (1).mp4,00:01:00,00:01:00,Близко стоит к путям при движении поезда
1,0000000_00000020240221082923_0001_IMP (1).mp4,00:06:00,06:22:00,Встал на подножку до полной остановки поезда
2,0000000_00000020240221082923_0001_IMP (1).mp4,00:10:00,00:10:00,Стоит возле поезда при стыковке
3,0000000_00000020240221082923_0001_IMP (1).mp4,00:10:00,00:11:00,Залез под поезд
4,0000000_00000020240221084423_0002_IMP.mp4,00:07:00,00:07:00,Близко стоит к путям при движении поезда
5,0000000_00000020240221084423_0002_IMP.mp4,00:08:00,00:08:00,"достает башмак, залезая под поезд"
6,0000000_00000020240221085923_0003_IMP.mp4,00:09:00,00:12:00,Встал на подножку до полной остановки поезда
7,0000000_00000020240221085923_0003_IMP.mp4,00:13:00,00:13:00,Стоит возле поезда при его движении
8,0000000_00000020240221085923_0003_IMP.mp4,00:13:00,00:13:00,Залез под поезд
9,0000000_00000020240508114229_0002.mp4,00:02:00,00:04:00,стоит слишком близко к поезду движущемуся


In [20]:
SAVING_FRAMES_PER_SECOND = 2 # кол-во кадров  в секунду
DELTA = 20 # отступаем до начала нарушения 5 сек

In [21]:
# info_df = info_df.tail()

In [22]:
info_df.head()

,videofile,start,end,description
0,0000000_00000020240221082923_0001_IMP (1).mp4,00:01:00,00:01:00,Близко стоит к путям при движении поезда
1,0000000_00000020240221082923_0001_IMP (1).mp4,00:06:00,06:22:00,Встал на подножку до полной остановки поезда
2,0000000_00000020240221082923_0001_IMP (1).mp4,00:10:00,00:10:00,Стоит возле поезда при стыковке
3,0000000_00000020240221082923_0001_IMP (1).mp4,00:10:00,00:11:00,Залез под поезд
4,0000000_00000020240221084423_0002_IMP.mp4,00:07:00,00:07:00,Близко стоит к путям при движении поезда


In [ ]:
save_result_path = 'images'


image_result_info = pd.DataFrame()

for vi, f in tqdm(enumerate(info_df.videofile.unique()), total=info_df.videofile.unique().shape[0]):
    temp = info_df[info_df.videofile==f]#.reset_index(drop=True)
    video_file = f'train/{f}'    
    file_names_df = main(video_file, f, vi, temp, DELTA, save_result_path)   
    image_result_info = pd.concat([image_result_info, file_names_df], ignore_index=True)

 12%|█▎        | 2/16 [07:03<46:16, 198.31s/it]  

In [ ]:
image_result_info.to_csv('image_result_info.csv')

In [ ]:
image_result_info.target.value_counts()

In [16]:
image_result_info[image_result_info.video_num==0]

,video_name,video_num,image_name,frame_duration,target,description
0,0000000_00000020240221082923_0001_IMP (1).mp4,0,video_0_frame_0-00-40.01_flag_0_violation_0.jpg,0-00-40.01,0,None
1,0000000_00000020240221082923_0001_IMP (1).mp4,0,video_0_frame_0-00-40.27_flag_0_violation_0.jpg,0-00-40.27,0,None
2,0000000_00000020240221082923_0001_IMP (1).mp4,0,video_0_frame_0-00-40.51_flag_0_violation_0.jpg,0-00-40.51,0,None
3,0000000_00000020240221082923_0001_IMP (1).mp4,0,video_0_frame_0-00-40.77_flag_0_violation_0.jpg,0-00-40.77,0,None
4,0000000_00000020240221082923_0001_IMP (1).mp4,0,video_0_frame_0-00-41.01_flag_0_violation_0.jpg,0-00-41.01,0,None
...,...,...,...,...,...,...
2316,0000000_00000020240221082923_0001_IMP (1).mp4,0,video_0_frame_0-14-59.03_flag_1_violation_1.jpg,0-14-59.03,1,Встал на подножку до полной остановки поезда
2317,0000000_00000020240221082923_0001_IMP (1).mp4,0,video_0_frame_0-14-59.27_flag_1_violation_1.jpg,0-14-59.27,1,Встал на подножку до полной остановки поезда
2318,0000000_00000020240221082923_0001_IMP (1).mp4,0,video_0_frame_0-14-59.53_flag_1_violation_1.jpg,0-14-59.53,1,Встал на подножку до полной остановки поезда
2319,0000000_00000020240221082923_0001_IMP (1).mp4,0,video_0_frame_0-14-59.77_flag_1_violation_1.jpg,0-14-59.77,1,Встал на подножку до полной остановки поезда


In [19]:
image_result_info.video_num.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 12, 13, 14, 15], dtype=int64)